# 신경망
**인공신경망(artificial neural network, 신경망)** 은 뇌의 동작에서 영감을 받은 예측 모델.

뇌는 뉴런의 집합, 각 뉴런은 다른 뉴런의 출력한 결과를 입력 받아 특정 연산을 수행, 계산 결과가 특정 임계치를 넘으면 활성화되고 넘지 않으면 활성화되지 않는다.

인공신경망은 인공적인 뉴런으로 구성, 입력값을 받아 계산을 수행, 신경망은 필기인식이나 얼굴인식 등 다양한 문제를 풀 수 있고 딥러닝에서 활발하게 활용되고 있다.

대부분의 신경망은 **블랙박스(black box)** 이기 때문에 그 속을 들여다본들 어떻게 문제를 풀고 있는지 제대로 이해할 수 없고 큰 신경망은 학습시키기도 어렵다. 언젠가 인공지능을 만들어 기술적 특이점(sigularity)을 앞당기고자 할 때 신경망은 좋은 선택일 수 있다.

## 18.1 퍼렙트론
**퍼셉트론(perceptron)** 은 n개의 이진수(binary)가 하나의 뉴런을 통과해서 가중 합이 0보다 크면 활성화되는 가장 단순한 신경망 구조.

In [2]:
from typing import List, Tuple

Vector = List[float]

def dot(v: Vector, w: Vector) -> float:
    """v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w),  "vectors must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v,w))

In [3]:
def step_function(x: float) -> float:
    return 1.0 if x >= 0 else 0.0

In [4]:
def perceptron_output(weights: Vector, bias: float, x: Vector) -> float:
    """퍼셉트론이 활성화되면 1, 아니면 0을 반환"""
    calculation = dot(weights, x) + bias
    return step_function(calculation)

퍼셉트론은 x를 초평면(hyperplane)으로 구분된 두 개의 공간으로 분리한다. (하나의 뉴런을 통과해서 가중 합이 0보다 크면 활성화 되기 때문에)

dot(weights, x) + bias == 0

가중치인 weights만 잘 선택되면 퍼셉트론으로도 여러 가지 간단한 문제를 풀 수 있다. 

e.g. 

두 개의 입력값이 모두 1이면 1을 반환, 하나라도 1이 아니면 0을 반환하는 'AND 게이트'를 만들 수 있다.ㅁ

In [6]:
and_weights = [2.,2]
and_bias = -3.

In [7]:
assert perceptron_output(and_weights, and_bias, [1,1]) == 1
assert perceptron_output(and_weights, and_bias, [0,1]) == 0
assert perceptron_output(and_weights, and_bias, [1,0]) == 0
assert perceptron_output(and_weights, and_bias, [0,0]) == 0

두 입력값이 모두 1이면 calculation이 2 + 2 - 3 = 1 이 되어 출력값은 1이다. 입력값 중 하나만 1이면 calculation은 2 + 0 - 3 = -1이 되어 출력값이 0이다. 입력값이 모두 0이면 calculation은 -3이 되어 출력값이 0이 된다. 

비슷한 방식으로 'OR 게이트'도 만들 수 있다.

In [8]:
or_weights = [2., 2]
or_bias = -1.

In [9]:
assert perceptron_output(or_weights, or_bias, [1,1]) == 1
assert perceptron_output(or_weights, or_bias, [0,1]) == 1
assert perceptron_output(or_weights, or_bias, [1,0]) == 1
assert perceptron_output(or_weights, or_bias, [0,0]) == 0

하나의 입력값을 받아 1을 0으로, 0을 1으로 변환하는 'NOT 게이트'도 만들 수 있다.

In [10]:
not_weights = [-2.]
not_bias =1.

In [11]:
assert perceptron_output(not_weights, not_bias, [0]) == 1
assert perceptron_output(not_weights, not_bias, [1]) == 0

위 AND gate는 1,1일 때 perceptron_output의 결과가 0보다 크게 하는 weights와 bias를 설정하면 된다. 

나머지 OR gate, NOT gate도 각각의 조건에 맞게 설정하면 된다.

그러나 단일 퍼셉트론만으로 풀 수 없는 문제도 많다. 

e.g.

아무리 고심해도 단일 퍼셉트론으로 둘 중 하나의 입력값이 1일 때 1을 반환하고 다른 모든 경우에는 0을 반환하는 'XOR 게이트'를 만들 수 없다. 

이런 문제를 풀려면 좀 더 복잡한 신경망이 필요

논리 게이트를 만들기 위해 반드시 뉴련을 사용해야 하는 것은 아니다.

In [12]:
and_gate = min
or_gate = max
xor_gate = lambda x, y: 0 if x == y else 1

우리 뇌의 실제 뉴런들이 그렇듯, 인공 뉴련 역시 서로 연결되었을 때 더욱 흥미로워진다. 

## 18.2 순방향 신경망
뇌의 구조는 매우 복잡하다. 인공신경망으로 뇌를 묘사할 때는 한 방향으로 연결된 **개별 층(layer)으로 추상화**하는 것이 **일반적**이다. 보통은 **입력값으로 받아 그대로 다음 층으로 값을 전송하는 입력층(input layer)**, **하나 이상의 은닉층(hidden layer)** 그리고 **최종값을 반환하는 출력층(output layer)** 등으로 구성한다.

퍼셉트론과 마찬가지로 (입력층에 속하지 않은) 각 뉴런에는 가중치와 편향이 있다. 여기서는 표현을 단순하게 하기 위해 편향을 가중치 벡터의 끝에 덧붙
인 후, 항상 1의 값을 가지는 편향의 입력값을 각 뉴련에 전달하자. 

각 뉴런은 퍼셉트론과 마찬가지로 입력값과 가중치의 곱을 합한다(dot product 수행). 하지만 여기서는 그 값에 step_function을 적용하는 것이 아니라, 조금 더 부드러운 모양을 가진 **시그모이드(sigmoid) 함수**를 적용해 볼 것이다.

In [15]:
import math

def sigmoid(t: float) -> float:
    return 1 / (1 + math.exp(-t))

단순한 step_function 대신 sigmoid를 사용하는 이유?

신경망을 학습하기 위해서는 미적분을 써야하고, 미적분을 사용하기 위해서는 **매끄러운 함수(smooth function)** 를 사용해야 하기 때문이다.

step_function은 연속적인 값을 가지지도 않지만, sigmoid는 그것을 잘 근사한(approximate) 매끄러운 곡선이다.

16장 '로지스틱 회귀 분석'에서 sigmoid 함수를 logistic이라고 불렀다. sigmoid와 logistic은 혼용해서 사용, 기술적으로 'sigmoid'는 함수의 모양을 지칭, 'logistic'은 함수 자체를 가리키는 말이다.

출력값은 아래와 같이 계산할 수 있다.

In [21]:
def neuron_output(weights: Vector, inputs: Vector) -> float:
    # weights에는 편향이 포함되어 있고, inputs는 1을 포함한다.
    return sigmoid(dot(weights, inputs))

이 함수가 주어지면 각 뉴런은 입력값의 수에 1이 추가된 길이의 (가중치 벡터에 편향을 추가했기 때문)벡터로 표현할 수 있다. 그러면 여러 뉴런으로 각 층을 구성하고, 여러 층으로 최종 신경망을 표현하면 된다.

즉, **신경망은 (여러) 리스트의 (뉴런의) 리스트의(가중치의) 벡터** 로 표현할 수 있다.

In [16]:
from typing import List

def feed_forward(neural_network: List[List[Vector]],
                input_vector: Vector) -> List[Vector]:
    """
    신경망에 입력 벡터를 전달한다. 
    (가장 마지막뿐 아니라) 모든 층의 결괏값을 반환한다.
    """
    outputs: List[Vector] = []
        
    for layer in neural_network:
        input_with_bias = input_vector + [1]              # 상수를 더한다.
        output = [neuron_output(neuron, input_with_bias)  # 결괏값을 
                 for neuron in layer]                     # 각 뉴런에 대해 계산한다.
        outputs.append(output)                            # 결괏값에 추가한다.
        
        # 이번 층의 결괏값은 다음 층의 입력값이 된다.
        input_vector = output
    return outputs

그럼 이제 단일 퍼셉트론으로는 만들 수 없었던 XOR 게이트를 만들어 보자.

neuron_output이 0 또는 1에 가까운 값을 가질 수 있도록 가중치의 크기를 조금 키워 주면 된다.

In [19]:
xor_network = [# 은닉층
                [[20., 20, -30], # 'and' 뉴런
                [20., 20, -10]], # 'or' 뉴런
               # 출력층
                [[-60., 60, -30]]]  # 1번째 입력값이 아닌 2번째 입력값을 받는 뉴런

In [20]:
# 순방향 신경망은 모든 층에 대한 결과를 계산하기 때문에
# [-1]은 최종결과를, [0]은 해당 벡터에서 값을 반환한다.
assert 0.000 < feed_forward(xor_network, [0, 0])[-1][0] < 0.001
assert 0.999 < feed_forward(xor_network, [1, 0])[-1][0] < 1.000
assert 0.999 < feed_forward(xor_network, [0, 1])[-1][0] < 1.000
assert 0.000 < feed_forward(xor_network, [1, 1])[-1][0] < 0.001

2차원 입력벡터를 받으면 은닉층은 두 입력값의 'and'와 'or'에 해당되는 값으로 구성된 2차원 벡터를 생성한다. 

그리고 출력층은 은닉층에서 받은 2차원 벡터에서 '첫 번째 입력값이 아닌 두 번째 입력값'을 계산해 준다. 그 결과는 'or이지만 and는 아닌' 네트워크, 즉 XOR이 된다. 

In [25]:
# feed_forward(xor_network, [0, 0])
# neural_network == xor_network, input_vector == [0, 0]
# input_with_bias = input_vector + [1] == [0, 0, 1]

neural_network = xor_network = [[[20., 20, -30], 
                                 [20., 20, -10]], 
                                [[-60., 60, -30]]]
input_vector = [0, 0]
outputs: List[Vector] = []

for layer in neural_network:
    input_with_bias = input_vector + [1]              # 상수를 더한다.
    output = [neuron_output(neuron, input_with_bias)  # 결괏값을 
             for neuron in layer]                     # 각 뉴런에 대해 계산한다.
    outputs.append(output)                            # 결괏값에 추가한다.